# CSA vs. OSA: An Introductive Study Replication

This notebook covers the replication of Table 1 and Table 3 of the following study: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4909617/#:~:text=The%20prevalence%20of%20CSA%20(defined,those%20aged%2065%20and%20older.&text=In%20a%20later%20cohort%20of,was%20appreciably%20higher%20(7.5%25). This was our first data task assigned by our sponsor to help us understand the goal of the project as well as important features to take note of. 

Table 1 describes demographics and sleep characteristics of the sleep study participants, classified into no sleep apnea, CSA, and OSA

Table 3 describes co-existing diseases and prescription drug use of sleep study participants.


## Replicating Table 1

In [1]:
import pandas as pd
import random
import numpy as np

random.seed(0)
np.random.seed(0)

df = pd.read_csv('../data/raw/shhs1-dataset-0.20.0.csv', encoding='cp1252', engine='python')

In [2]:
table1 = ['bmi_s1', 'age_s1', 'gender', 'systbp', 'diasbp', 'ess_s1', 'ahi_a0h4', 'ahi_c0h4', 'ahi_o0h4']

In [3]:
df = df[table1]

In [4]:
df.dropna(inplace=True)

Diagnosis of Sleep Apnea:

* Obstructive sleep apnea: # of total apnea events >= 5 & obstructive AHI > central AHI
* Central sleep apnea: # of central apnea events >= 5 & central AHI > obstructive AHI
* no central sleep apnea: # of total apnea events < 5

In [16]:
# OSA: TOTAL AHI >= 5 & OAHI > CAHI
osa = df[(df['ahi_a0h4'] >= 5) & (df['ahi_o0h4'] > df['ahi_c0h4'])]
# CSA: CAHI >= 5 & CAHI > OAHI
csa = df[(df['ahi_c0h4'] >= 5) & (df['ahi_c0h4'] > df['ahi_o0h4'])]
# No SA: TOTAL AHI < 5
no_sa = df[df['ahi_a0h4'] < 5]

In [18]:
# split csa into bins: [0, 5, 15, 30, 1000]
csa.loc[:, 'ahi_c0h4'] = pd.cut(csa['ahi_c0h4'], bins=[0, 5, 15, 30, 1000], labels=['0-5', '5-15', '15-30', '30+'])

# count the number of each bin
csa['ahi_c0h4'].value_counts()

ahi_c0h4
5-15     97
15-30    36
30+      22
0-5       0
Name: count, dtype: int64

In [19]:
print(len(no_sa))
print(len(osa))
print(len(csa))

2649
2503
155


In [21]:
# add labels based on the criteria above
osa.loc[:, 'label'] = 'OSA'
csa.loc[:, 'label'] = 'CSA'
no_sa.loc[:, 'label'] = 'No SA'

# combine the three dataframes
df = pd.concat([osa, csa, no_sa])

In [22]:
features = ['bmi_s1', 'age_s1', 'gender', 'systbp', 'diasbp', 'ess_s1', 'ahi_a0h4']

In [28]:
df_table1 = df[features + ['label']]

In [29]:
df_table1

,bmi_s1,age_s1,gender,systbp,diasbp,ess_s1,ahi_a0h4,label
1,32.950680,78,1,168.0,68.0,14.0,19.780220,OSA
2,24.114150,77,2,127.0,68.0,5.0,5.020921,OSA
6,29.983588,52,1,142.0,99.0,11.0,10.105263,OSA
8,25.817447,69,1,201.0,101.0,10.0,24.409673,OSA
11,25.401235,68,1,152.0,90.0,7.0,20.417335,OSA
...,...,...,...,...,...,...,...,...
5795,35.790598,71,2,126.0,73.0,5.0,2.284041,No SA
5796,21.957367,55,2,136.0,77.0,13.0,0.807537,No SA
5798,32.414213,54,2,118.0,66.0,7.0,1.878669,No SA
5801,24.228571,55,1,89.0,56.0,17.0,3.605769,No SA


In [32]:
df_table1.loc[:,:].dropna(inplace=True)

In [33]:
df_table1

,bmi_s1,age_s1,gender,systbp,diasbp,ess_s1,ahi_a0h4,label
1,32.950680,78,1,168.0,68.0,14.0,19.780220,OSA
2,24.114150,77,2,127.0,68.0,5.0,5.020921,OSA
6,29.983588,52,1,142.0,99.0,11.0,10.105263,OSA
8,25.817447,69,1,201.0,101.0,10.0,24.409673,OSA
11,25.401235,68,1,152.0,90.0,7.0,20.417335,OSA
...,...,...,...,...,...,...,...,...
5795,35.790598,71,2,126.0,73.0,5.0,2.284041,No SA
5796,21.957367,55,2,136.0,77.0,13.0,0.807537,No SA
5798,32.414213,54,2,118.0,66.0,7.0,1.878669,No SA
5801,24.228571,55,1,89.0,56.0,17.0,3.605769,No SA


In [38]:
# Do statistics based on label, Do Mean, Std, and perbalance(95% CI)
df_table1.groupby('label').describe().T

label                      CSA        No SA          OSA
BMI          count  155.000000  2649.000000  2503.000000
             mean    29.437723    26.808917    29.338060
             std      4.883141     4.403147     5.342028
             min     21.023138    18.000000    18.000000
             25%     25.801037    23.828125    25.722984
             50%     28.509508    26.264784    28.675689
             75%     32.542248    29.233480    32.111039
             max     46.650769    50.000000    50.000000
Age          count  155.000000  2649.000000  2503.000000
             mean    64.393548    61.050208    65.714343
             std     11.254146    11.164797    10.453291
             min     40.000000    39.000000    39.000000
             25%     56.000000    53.000000    58.000000
             50%     66.000000    60.000000    66.000000
             75%     74.000000    70.000000    74.000000
             max     90.000000    90.000000    90.000000
Gender       count  155.000000  2649.000000  2503.000000
             mean     1.303226     1.647414     1.407511
             std      0.461142     0.477865     0.491470
             min      1.000000     1.000000     1.000000
             25%      1.000000     1.000000     1.000000
             50%      1.000000     2.000000     1.000000
             75%      2.000000     2.000000     2.000000
             max      2.000000     2.000000     2.000000
Systolic BP  count  155.000000  2649.000000  2503.000000
             mean   129.800000   124.765949   129.500200
             std     18.958053    19.060673    19.149771
             min     89.000000    84.000000    52.000000
             25%    117.000000   111.000000   116.000000
             50%    128.000000   122.000000   128.000000
             75%    140.000000   136.000000   141.000000
             max    188.000000   214.000000   210.000000
Diastolic BP count  155.000000  2649.000000  2503.000000
             mean    75.529032    73.028690    74.041550
             std     11.791098    11.022701    12.092696
             min     49.000000    31.000000    10.000000
             25%     67.000000    66.000000    66.000000
             50%     74.000000    72.000000    74.000000
             75%     84.000000    80.000000    82.000000
             max    110.000000   113.000000   132.000000
ESS          count  155.000000  2649.000000  2503.000000
             mean     8.058065     7.303888     8.230124
             std      4.859371     4.244526     4.489594
             min      0.000000     0.000000     0.000000
             25%      4.000000     4.000000     5.000000
             50%      8.000000     7.000000     8.000000
             75%     11.000000    10.000000    11.000000
             max     22.000000    24.000000    24.000000
AHI          count  155.000000  2649.000000  2503.000000
             mean    18.455492     1.983487    18.475090
             std     14.753549     1.417913    15.641994
             min      5.078597     0.000000     5.000000
             25%      8.123581     0.738916     8.178284
             50%     12.334495     1.714286    13.353116
             75%     22.404958     3.083512    22.503651
             max     79.833333     4.990548   154.285714

In [39]:
# Rename the columns
df_table1.loc[:,:].rename({'bmi_s1': 'BMI', 'age_s1': 'Age', 'gender': 'Gender', 'systbp': 'Systolic BP', 'diasbp': 'Diastolic BP', 'ess_s1': 'ESS', 'ahi_a0h4': 'AHI'}, axis=1, inplace=True)

# Keep only mean, std, and 95% CI
# round to 2 decimal places
means = df_table1.groupby('label').mean().T.round(1)
stds = df_table1.groupby('label').std().T.round(1)
counts = df_table1.groupby('label').count().T

In [40]:
counts

label,CSA,No SA,OSA
BMI,155,2649,2503
Age,155,2649,2503
Gender,155,2649,2503
Systolic BP,155,2649,2503
Diastolic BP,155,2649,2503
ESS,155,2649,2503
AHI,155,2649,2503


In [41]:
means

label,CSA,No SA,OSA
BMI,29.4,26.8,29.3
Age,64.4,61.1,65.7
Gender,1.3,1.6,1.4
Systolic BP,129.8,124.8,129.5
Diastolic BP,75.5,73.0,74.0
ESS,8.1,7.3,8.2
AHI,18.5,2.0,18.5


In [42]:
stds

label,CSA,No SA,OSA
BMI,4.9,4.4,5.3
Age,11.3,11.2,10.5
Gender,0.5,0.5,0.5
Systolic BP,19.0,19.1,19.1
Diastolic BP,11.8,11.0,12.1
ESS,4.9,4.2,4.5
AHI,14.8,1.4,15.6


In [43]:
# Comparisons of continuous variables were made among 
# the 3 groups with one-way ANOVA with subsequent pairwise 
# Tukey HSD test
from scipy import stats

# ANOVA
# H0: The means of the groups are equal
# H1: The means of the groups are not equal
# p-value < 0.05, reject H0
# p-value > 0.05, fail to reject H0
# p-value = 0.05, marginal

# BMI
print("BMI")
print(stats.f_oneway(osa['bmi_s1'], csa['bmi_s1'], no_sa['bmi_s1']))

# Age
print("Age")
print(stats.f_oneway(osa['age_s1'], csa['age_s1'], no_sa['age_s1']))

# gender
print("Gender")
print(stats.f_oneway(osa['gender'], csa['gender'], no_sa['gender']))

# systbp
print("Systbp")
print(stats.f_oneway(osa['systbp'], csa['systbp'], no_sa['systbp']))

# diasbp
print("Diasbp")
print(stats.f_oneway(osa['diasbp'], csa['diasbp'], no_sa['diasbp']))

# ess_s1
print("Ess_s1")
print(stats.f_oneway(osa['ess_s1'], csa['ess_s1'], no_sa['ess_s1']))

# ahi_a0h4
print("Ahi_a0h4")
print(stats.f_oneway(osa['ahi_a0h4'], csa['ahi_a0h4'], no_sa['ahi_a0h4']))

BMI
F_onewayResult(statistic=178.89195549666258, pvalue=6.55236171672407e-76)
Age
F_onewayResult(statistic=119.9245383463436, pvalue=1.1499882927179695e-51)
Gender
F_onewayResult(statistic=174.83909338791827, pvalue=2.9274338921080018e-74)
Systbp
F_onewayResult(statistic=41.076643041870014, pvalue=1.983292012261657e-18)
Diasbp
F_onewayResult(statistic=7.208300939239418, pvalue=0.0007476898168833154)
Ess_s1
F_onewayResult(statistic=29.133155965242462, pvalue=2.60988694668077e-13)
Ahi_a0h4
F_onewayResult(statistic=1469.7275857978243, pvalue=0.0)


## Table 3

In [44]:
import numpy as np
import pandas as pd

In [70]:
shhs1 = pd.read_csv('../data/raw/shhs1-dataset-0.20.0.csv', encoding='cp1252', engine='python')

In [71]:
# Get the No SA, OSA, CSA-G, and CSR people
no_sa = shhs1[shhs1['ahi_a0h4'] < 5]
len(no_sa)

2830

In [72]:
osa = shhs1[shhs1['ahi_a0h4'] >= 5]
osa = osa[osa['ahi_o0h4'] > osa['ahi_c0h4']]
len(osa)

2632

In [73]:
csa = shhs1[shhs1['ahi_c0h4'] >= 5]
csa = csa[csa['ahi_c0h4'] > csa['ahi_o0h4']]
len(csa)

165

In [39]:
# Since we don't have the data for periodic breathing (pb), there's no Cheyne-Stokes respiration group data (CSR), a column in Table 3

In [74]:
variables = ['mi15', 'angina15',
'stroke15','cabg15',
'pacem15', 'copd15',
'asthma15', 'loop1', 'diuret1', 
'ccb1', 'beta1', 'ace1', 
'lipid1', 'ohga1',
'warf1', 'asa1', 'nsaid1',
'benzod1'
]

In [75]:
# Define a dictionary to save the result
result = dict()
for column in variables:
    # Define the specific value we want to calculate the percentage for
    specific_value = 1

    # Calculate the percentage of the specific value in the column
    percentage = (no_sa[column] == specific_value).mean() * 100

    # Step 3: Calculate the standard error and margin of error for the percentage
    n = len(no_sa)
    standard_error = np.sqrt((percentage / 100 * (1 - percentage / 100)) / n)
    margin_of_error = 1.96 * standard_error  # 1.96 corresponds to a 95% confidence interval

    # Step 4: Calculate the confidence interval
    lower_bound = percentage - margin_of_error
    upper_bound = percentage + margin_of_error
    result[column] = [percentage, lower_bound, upper_bound]

In [76]:
# Convert it into dataframe
no_sa_result = pd.DataFrame(result).T
no_sa_result.rename(columns={0:'No SA Percentage', 1:'No SA lower_bound', 2:'No SA upper_bound'}, inplace=True)

In [77]:
# Define a dictionary to save the result
result = dict()
for column in variables:
    # Define the specific value we want to calculate the percentage for
    specific_value = 1

    # Calculate the percentage of the specific value in the column
    percentage = (osa[column] == specific_value).mean() * 100

    # Step 3: Calculate the standard error and margin of error for the percentage
    n = len(no_sa)
    standard_error = np.sqrt((percentage / 100 * (1 - percentage / 100)) / n)
    margin_of_error = 1.96 * standard_error  # 1.96 corresponds to a 95% confidence interval

    # Step 4: Calculate the confidence interval
    lower_bound = percentage - margin_of_error
    upper_bound = percentage + margin_of_error
    result[column] = [percentage, lower_bound, upper_bound]
    
    

In [78]:
# Convert it into dataframe
osa_result = pd.DataFrame(result).T
osa_result.rename(columns={0:'OSA Percentage', 1:'OSA lower_bound', 2:'OSA upper_bound'}, inplace=True)

In [79]:
# Define a dictionary to save the result
result = dict()
for column in variables:
    # Define the specific value we want to calculate the percentage for
    specific_value = 1

    # Calculate the percentage of the specific value in the column
    percentage = (csa[column] == specific_value).mean() * 100

    # Step 3: Calculate the standard error and margin of error for the percentage
    n = len(no_sa)
    standard_error = np.sqrt((percentage / 100 * (1 - percentage / 100)) / n)
    margin_of_error = 1.96 * standard_error  # 1.96 corresponds to a 95% confidence interval

    # Step 4: Calculate the confidence interval
    lower_bound = percentage - margin_of_error
    upper_bound = percentage + margin_of_error
    result[column] = [percentage, lower_bound, upper_bound]
    

In [80]:
# Convert it into dataframe
csa_result = pd.DataFrame(result).T
csa_result.rename(columns={0:'CSA Percentage', 1:'CSA lower_bound', 2:'CSA upper_bound'}, inplace=True)

In [81]:
no_sa_result

,No SA Percentage,No SA lower_bound,No SA upper_bound
mi15,4.628975,4.621234,4.636717
angina15,5.512367,5.503959,5.520776
stroke15,2.402827,2.397185,2.408469
cabg15,2.650177,2.644259,2.656095
pacem15,0.353357,0.351171,0.355543
copd15,1.166078,1.162122,1.170033
asthma15,9.081272,9.070685,9.091859
loop1,2.791519,2.785450,2.797589
diuret1,13.003534,12.991141,13.015926
ccb1,11.201413,11.189794,11.213033


In [82]:
osa_result

,OSA Percentage,OSA lower_bound,OSA upper_bound
mi15,7.408815,7.399165,7.418464
angina15,8.510638,8.500357,8.520919
stroke15,4.027356,4.020112,4.034599
cabg15,4.293313,4.285845,4.300782
pacem15,1.519757,1.515249,1.524264
copd15,0.987842,0.984198,0.991486
asthma15,7.902736,7.892796,7.912675
loop1,5.965046,5.956320,5.973772
diuret1,18.351064,18.336802,18.365325
ccb1,16.641337,16.627615,16.655060


In [49]:
csa_result

,CSA Percentage,CSA lower_bound,CSA upper_bound
mi15,9.696970,9.686069,9.707870
angina15,14.545455,14.532467,14.558442
stroke15,4.242424,4.235000,4.249849
cabg15,8.484848,8.474584,8.495113
pacem15,3.030303,3.023988,3.036618
copd15,1.212121,1.208090,1.216152
asthma15,9.696970,9.686069,9.707870
loop1,12.727273,12.714996,12.739550
diuret1,21.212121,21.197062,21.227181
ccb1,20.606061,20.591161,20.620960


In [83]:
new_index_mapping = {'mi15': 'History of MI', 'angina15': 'Angina', 'stroke15': 'History of stroke', 'cabg15': 'History of CABG',
                    'pacem15': 'History of pacemaker', 'copd15': 'History of COPD', 'asthma15': 'History of Asthma',
                    'loop1': 'Loop diuretic', 'diuret1': 'Any diuretic', 'ccb1': 'Calcium channel blocker', 'beta1': 'Beta blocker',
                    'ace1': 'Ace inhibitor', 'lipid1': 'Anti-lipid', 'ohga1': 'Oral Hypoglycemic', 'warf1': 'Warfarin',
                    'asa1': 'Aspirin', 'nsaid1': 'Non-aspirin NSAID', 'benzod1': 'Benzodiazepine'}
csa_result = csa_result.rename(index=new_index_mapping)
osa_result = osa_result.rename(index=new_index_mapping)
no_sa_result = no_sa_result.rename(index=new_index_mapping)

In [84]:
csa_result

,CSA Percentage,CSA lower_bound,CSA upper_bound
History of MI,9.696970,9.686067,9.707872
Angina,14.545455,14.532465,14.558444
History of stroke,4.242424,4.234998,4.249850
History of CABG,8.484848,8.474582,8.495115
History of pacemaker,3.030303,3.023987,3.036619
History of COPD,1.212121,1.208090,1.216153
History of Asthma,9.696970,9.686067,9.707872
Loop diuretic,12.727273,12.714994,12.739552
Any diuretic,21.212121,21.197059,21.227183
Calcium channel blocker,20.606061,20.591158,20.620963


In [85]:
combined_df = pd.concat([no_sa_result, osa_result], axis=1)
combined_df = pd.concat([combined_df, csa_result], axis=1)

In [86]:
combined_df

,No SA Percentage,No SA lower_bound,No SA upper_bound,OSA Percentage,OSA lower_bound,OSA upper_bound,CSA Percentage,CSA lower_bound,CSA upper_bound
History of MI,4.628975,4.621234,4.636717,7.408815,7.399165,7.418464,9.696970,9.686067,9.707872
Angina,5.512367,5.503959,5.520776,8.510638,8.500357,8.520919,14.545455,14.532465,14.558444
History of stroke,2.402827,2.397185,2.408469,4.027356,4.020112,4.034599,4.242424,4.234998,4.249850
History of CABG,2.650177,2.644259,2.656095,4.293313,4.285845,4.300782,8.484848,8.474582,8.495115
History of pacemaker,0.353357,0.351171,0.355543,1.519757,1.515249,1.524264,3.030303,3.023987,3.036619
History of COPD,1.166078,1.162122,1.170033,0.987842,0.984198,0.991486,1.212121,1.208090,1.216153
History of Asthma,9.081272,9.070685,9.091859,7.902736,7.892796,7.912675,9.696970,9.686067,9.707872
Loop diuretic,2.791519,2.785450,2.797589,5.965046,5.956320,5.973772,12.727273,12.714994,12.739552
Any diuretic,13.003534,12.991141,13.015926,18.351064,18.336802,18.365325,21.212121,21.197059,21.227183
Calcium channel blocker,11.201413,11.189794,11.213033,16.641337,16.627615,16.655060,20.606061,20.591158,20.620963


In [87]:
df = combined_df[['No SA Percentage', 'OSA Percentage', 'CSA Percentage']]

In [88]:
df

,No SA Percentage,OSA Percentage,CSA Percentage
History of MI,4.628975,7.408815,9.696970
Angina,5.512367,8.510638,14.545455
History of stroke,2.402827,4.027356,4.242424
History of CABG,2.650177,4.293313,8.484848
History of pacemaker,0.353357,1.519757,3.030303
History of COPD,1.166078,0.987842,1.212121
History of Asthma,9.081272,7.902736,9.696970
Loop diuretic,2.791519,5.965046,12.727273
Any diuretic,13.003534,18.351064,21.212121
Calcium channel blocker,11.201413,16.641337,20.606061
